# Sistemas Inteligentes (2022/2023)

## Avaliação contínua 3 
## O puzzle Minesweeper

### Prazo de entrega: 28 de Abril

<img src="imagens/AbleAcidicArabianhorse-size_restricted.gif" width=150 height=150 />

## Puzzle Minesweeper
No jogo do `Minesweeper` queremos determinar quais as casas de um tabuleiro 2D que contêm minas. Em qualquer instante estamos perante um tabuleiro parcialmente revelado como o que está em baixo:

<img src="imagens/minesweeperStandard.png" width=100 height=100/>

Os quadrados (ou células) podem estar marcados ou escondidos (não marcados). Os escondidos podem estar minados ou limpos de minas. Os marcados, limpos de minas, indicam o número de minas presentes nas casas imediatamente adjacentes (ortogonais e diagonais). Em cada passo do jogo escolhemos uma casa vazia (quadrado escondido) para ser revelada. Ganha-se quando revelamos todas as casas limpas de minas sem revelar uma mina. Podemos usar CSPs para obter informação sobre as casas desconhecidas em cada instante. No entanto, só poderemos obter informação sobre as casas que forem vizinhas de casas marcadas.

## Objetivos

Na formulação N-ária, as variáveis seriam apenas as células não marcadas que são vizinhas das células marcadas, que são as únicas que correspondem à solução do puzzle. Os domínios poderiam ser bits que indicam se há bomba (1) ou não (0). A cada célula marcada corresponde uma restrição que indica que a sua etiqueta corresponde exactamente à soma dos valores das variáveis correspondentes às células vizinhas. A classe CSP do `aima-python` que lida com a formulação de problemas de satisfação de restrições só lida com restrições binárias.

Neste projecto deseja-se **formular uma determinada configuração de um puzzle de minesweeper como um CSP binário**. Para isso vamos utilizar a classe CSP do aima-python que está implementada no `csp.py`, tendo que criar uma instância dessa classe para qualquer puzzle Minesweeper dado como input. Terão de implementar a função ***minesweeper_CSP*** que recebe uma representação de um qualquer puzzle e devolve uma instância da classe CSP.
```python
def minesweeper_CSP(puzz):
    pass
```

A formulação não será propriamente livre: vamos impor o que serão as variáveis e a forma como são apresentados os domínios e como representamos um puzzle minesweeper que queremos resolver. Fornecemos a função ***puzzle_display*** que permite imprimir em modo texto um determinado puzzle antes e depois de ser resolvido.

## Representação de Puzzles Minesweeper
Vamos representar um puzzle por resolver, o input da função ***minesweeper_CSP***, por uma lista de listas representando cada linha, em que as células sem bomba e que estão marcadas serão inteiros e as células escondidas serão #s.

O puzzle seguinte:

```python
1 2 # # # # #
1 # # # 2 # #
# # # 4 # # #
# # 2 # 3 # #
2 2 # 2 # # #
# # # # # # #
```

será representado por:



In [5]:
puzzle=[[1, 2,'#','#','#','#','#'],\
        [1,'#','#','#',2,'#','#'],\
        ['#','#','#',4,'#','#','#'],\
        ['#','#',2,'#',3,'#','#'],\
        [2,2,'#',2,'#','#','#'],\
        ['#','#','#','#','#','#','#']] 

#### Apresentação dos puzzles em texto: a função ***display***

In [9]:
def puzzle_display(puzzle):
    compy=len(puzzle)
    compx=len(puzzle[0])
    for y in range(compy):
        for x in range(compx):
            print(puzzle[y][x],end=' ')
        print()

In [7]:
puzzle_display(puzzle)

1 2 # # # # # 
1 # # # 2 # # 
# # # 4 # # # 
# # 2 # 3 # # 
2 2 # 2 # # # 
# # # # # # # 


Esse mesmo puzzle resolvido será representado assim:

In [11]:
puzzle_fim=[[1, 2, '~', '~', '~', '~', '#'],\
            [1, '@', '@', '~', 2, '~', '#'],\
            ['~', '~', '@', 4, '@', '@', '#'],\
            ['~', '~', 2, '@', 3, '~', '#'],\
            [2, 2, '~', 2, '~', '~', '#'],\
            ['@', '@', '~', '@', '~', '#', '#']]
puzzle_display(puzzle_fim)

1 2 ~ ~ ~ ~ # 
1 @ @ ~ 2 ~ # 
~ ~ @ 4 @ @ # 
~ ~ 2 @ 3 ~ # 
2 2 ~ 2 ~ ~ # 
@ @ ~ @ ~ # # 


Reparem que todas as células adjacentes a células marcadas ficaram destapadas (ou são células vazias, representadas por '~' ou são bombas, representadas por '@'). As restantes células não contribuem para a solução do CSP.

Após a aplicação do `backtracking_search` para resolver uma instância do puzzle Minesweeper, podem usar a função ***fill_puzzle*** que dado o puzzle original e uma afetação `r` retorna a lista de listas correspondente à solução do puzzle.
```python
from copy import deepcopy 
def fill_puzzle(puzzle,sol):
    copia=deepcopy(puzzle)
    for s in sol:
        if isinstance(sol[s],int):
            xy=s.split('_')
            x=int(xy[1])
            y=int(xy[2])
            copia[y][x]='@' if sol[s] else '~'
    return copia
```
Assim o output da função ***fill_puzzle*** é
```python
fill_puzzle(puzzle,r)
```
```
[[1, 2, '~', '~', '~', '~', '#'],
 [1, '@', '@', '~', 2, '~', '#'],
 ['~', '~', '@', 4, '@', '@', '#'],
 ['~', '~', 2, '@', 3, '~', '#'],
 [2, 2, '~', 2, '~', '~', '#'],
 ['@', '@', '~', '@', '~', '#', '#']]
```
e a solução do puzzle pode ser visualizada com a função ***puzzle_display*** como apresentado anteriormente.

## Formulação num grafo binário

### Variáveis
Nós sabemos que as células do puzzle que estão escondidas poderão ter bombas ou não e que as células marcadas não têm bombas de certeza. Teremos variáveis 'V_X_Y' para as células correspondentes do puzzle de dimensões MxN (M linhas e N colunas) desde que essas células (X,Y) sejam marcadas (1..8), ou sejam não marcadas mas vizinhas de células marcadas. As células não marcadas mas vizinhas de células marcadas também existem na versão N-ária. 

Vamos seguir o processo de conversão simples de modelizações não binárias em binárias (ver um exemplo de conversão na imagem seguinte e ver também as soluções do exercício do minesweeper da TP6). 

<img src="imagens/ExemploconversaoCSPbinarios.PNG" width=400 height=400 /> 


Assim, na formulação binária a cada restrição vai corresponder uma nova variável e como a cada célula marcada corresponde um restrição, teremos novas variáveis 'V_X_Y' para cada célula (X,Y) com a etiqueta do número de bombas das vizinhas. Podemos ter células do puzzle que após a procura da solução continuamos a nada saber sobre elas. Serão as células não marcadas que não são vizinhas imediatas de células marcadas.  

Vamos ordenar as dimensões de cima para baixo e da direita para a esquerda, considerando que a célula de topo esquerdo tem coordenadas (0,0) e a célula do fundo à direita tem as coordenadas (N,M). 

Para o caso do puzzle seguinte:
```python
1 2 # # # # #
1 # # # 2 # #
# # # 4 # # #
# # 2 # 3 # #
2 2 # 2 # # #
# # # # # # # 
```
cada uma das células marcadas com V corresponderiam a variáveis, desde 'V_0_0' até 'V_4_5':

```python
V_0_0  V_1_0  V_2_0  V_3_0  V_4_0  V_5_0   #
V_0_1  V_1_1  V_2_1  V_3_1  V_4_1  V_5_1   #
V_0_2  V_1_2  V_2_2  V_3_2  V_4_2  V_5_2   #
V_0_3  V_1_3  V_2_3  V_3_3  V_4_3  V_5_3   #
V_0_4  V_1_4  V_2_4  V_3_4  V_4_4  V_5_4   #
V_0_5  V_1_5  V_2_5  V_3_5  V_4_5    #     # 
```

Temos 35 variáveis, havendo 7 células do puzzle sem correspondência com nenhuma variável e que não farão parte da solução.

**O atributo `variables` da instância do CSP criada terá que estar ordenado alfanumericamente.**

### Domínios

Os domínios, para usarmos a classe CSP do `aima-python`, têm sempre de ser dicionários em que as chaves são as variáveis e os valores do dicionários são listas de valores que as variáveis podem tomar. Têm, no entanto, a liberdade de definirem a estrutura de dados que acharem mais adequada para os valores dos domínios, mas para os testes automáticos terão de eventualmente converter esses valores em bits e tuplos de bits.

#### Domínios das variáveis não marcadas
Para as células não marcadas, que são vizinhas de células marcadas, os domínios terão de ser vistos como bits nos testes automáticos: ou 0 ou 1. O 0 indica ausência de bomba e o 1 a sua presença. A ordem dos valores na lista tem de ser sempre $[0, 1] $

#### Domínios das variáveis marcadas
Como dissemos atrás não impomos uma estrutura de dados para os valores que as variáveis podem tomar. No entanto a ordem dos domínios, que são listas de valores, tem de seguir uma ordem específica. Para os testes automáticos, terão de converter esses valores em tuplos de bits sendo decisiva a posição dos seus elementos. Nesses tuplos, devem respeitar o sentido dos ponteiros do relógio (ou sentido horário) dos vizinhos das células marcadas, começando a norte e acabando a noroeste. A lista de valores tem de vir também ordenada por ordem crescente de acordo com a ordenação de tuplos.

Notem que embora tenham a liberdade de definirem a estrutura de dados para os valores dos domínios, a lista de valores para cada chave do dicionário tem forçosamente de estar ordenada pela mesma ordem que exigimos na conversão. Só assim garantimos nos testes automáticos que ao resolvermos um CSP a procura segue sempre a mesma ordem de afectações de valores às variáveis.


Notem que esses domínios terão que estar filtrados de modo a satisfazerem a restrição unária correspondente. 

Para o caso do puzzle seguinte:
```python
1 2 # # # # #
1 # # # 2 # #
# # # 4 # # #
# # 2 # 3 # #
2 2 # 2 # # #
# # # # # # # 
```
Teríamos 10 restrições uma por cada célula marcada, em que, por exemplo, 3 delas são as seguintes:

    'V_1_1' = 1
    'V_2_0' + 'V_2_1' + 'V_1_1' = 2
    'V_3_1' + 'V_4_2' + 'V_3_3' + 'V_2_2' + 'V_2_1' = 4
    ...

A restrição 'V_1_1'=1 corresponderá à variável 'V_0_0' que após a conversão que falámos terá um tuplo unitário (só tem um vizinho sem marca), para valor: (1,)

A restrição 'V_2_0'+'V_2_1'+'V_1_1'=2 corresponderá à variável 'V_1_0' que após a conversão que falámos terá que ter o seguinte domínio: 
```python 
[(0,1,1),(1,0,1),(1,1,0)]
```
Notem que no triplo (a,b,c), teremos que respeitar o sentido horário das células vizinhas de 'V_1_0': 'a' corresponde à variável 'V_2_0', 'b' à variável 'V_2_1' e 'c' à variável 'V_1_1'.  

A restrição 'V_3_1'+'V_4_2'+'V_3_3'+'V_2_2'+'V_2_1'+=4 corresponderá à variável 'V_3_2' que após a conversão que falámos terá que ter o seguinte domínio: 
```python 
[(0,1,1,1,1),(1,0,1,1,1),(1,1,0,1,1),(1,1,1,0,1),(1,1,1,1,0)]
```
Notem que no triplo (a,b,c,d,e), teremos que respeitar o sentido horário das células vizinhas de 'V_3_2': 'a' corresponde à variável 'V_3_1', 'b' à variável 'V_4_2', 'c' à variável 'V_3_3', 'd' à variável 'V_2_2' e 'e' à variável 'V_2_1'.

**Os domínios de cada variável no atributo `domains` da instância do CSP criada terão de estar ordenados de forma crescente de valores,** i.e., deve primeiro testar-se o 0 e depois o 1. Notem que nos exemplos apresentados os domínios, após conversão, de 'V_1_0' e de 'V_3_2' estão ordenados de forma crescente. Por exemplo,
```
{'V_5_3': [0, 1],
 'V_0_0': [(1,)],
 'V_1_0': [(0, 1, 1), (1, 0, 1), (1, 1, 0)],
 'V_0_1': [(0, 0, 1), (0, 1, 0), (1, 0, 0)],
 ...
}
```

#### Função ***show_domains***

Para os testes automáticos, terão que desenvolver uma função ***show_domains*** que recebe um dicionário com os domínios e retorna uma lista com os domínios pronta para ser testada automaticamente, i.e., com a **lista de valores ordenada de forma crescente tanto em termos de chaves como de valores**.
```python
def show_domains(dom):
    pass
```
Por exemplo,
```
[('V_0_0', [(1,)]),
 ('V_0_1', [(0, 0, 1), (0, 1, 0), (1, 0, 0)]),
 ('V_0_2', [0, 1]),
 ('V_0_3', [0, 1]),
 ('V_0_4',
  [(0, 0, 1, 1),
   (0, 1, 0, 1),
   (0, 1, 1, 0),
   (1, 0, 0, 1),
   (1, 0, 1, 0),
   (1, 1, 0, 0)]),
   ...
]
```

### Vizinhança
O grafo de vizinhança das variáveis terá de cumprir os requisitos da classe CSP: um dicionário em que as chaves são as variáveis e os valores são listas de variáveis. Não é necessário impor nenhuma ordem nem nas chaves nem nos vizinhos porque os nossos testes automáticos podem verificar se o grafo está bem montado independentemente da ordem de criação das chaves e da ordem dos vizinhos na lista.

### Restrições
Não impomos nenhuma forma de implementação da função que faz o teste de satisfação de restrições entre dois valores de duas variáveis. Vamos apenas de um modo pedagógico indicar que se os valores dos domínios fossem os bits e tuplos referidos em cima, as restrições teriam que verificar apenas a igualdade entre os valores das variáveis cobertas de domínios binários e as correspondentes posições dos valores das variáveis das células marcadas vizinhas.

Assim, damos exemplos de algumas restrições que essa função teria de levar em conta:
```python
'V_1_1'='V_0_0'[0]

'V_2_0'='V_1_0'[0]
'V_2_1'='V_1_0'[1]
'V_1_1'='V_1_0'[2]

'V_3_1'='V_3_2'[0]
'V_4_2'='V_3_2'[1]
'V_3_3'='V_3_2'[2]
'V_2_2'='V_3_2'[3]
'V_2_1'='V_3_2'[4]
...
```

## Exemplo

Considere o puzzle Minesweeper do inicio do enunciado
```
1 2 # # # # # 
1 # # # 2 # # 
# # # 4 # # # 
# # 2 # 3 # # 
2 2 # 2 # # # 
# # # # # # # 
```

Vamos aplicar o função **minesweeper_CSP** desenvolvida e validar os outputs.

As variáveis deve ser um lista ordenada da seguinte forma:
```
['V_0_0', 'V_0_1', 'V_0_2', 'V_0_3', 'V_0_4', 'V_0_5', 'V_1_0', 'V_1_1', 'V_1_2', 'V_1_3', 'V_1_4', 'V_1_5', 'V_2_0', 'V_2_1', 'V_2_2', 'V_2_3', 'V_2_4', 'V_2_5', 'V_3_0', 'V_3_1', 'V_3_2', 'V_3_3', 'V_3_4', 'V_3_5', 'V_4_0', 'V_4_1', 'V_4_2', 'V_4_3', 'V_4_4', 'V_4_5', 'V_5_0', 'V_5_1', 'V_5_2', 'V_5_3', 'V_5_4']
```

Os domínios deve ser uma lista de valores ordenada dada pela função **show_domains** que terão de desenvolver:
```
[('V_0_0', [(1,)]), 
 ('V_0_1', [(0, 0, 1), (0, 1, 0), (1, 0, 0)]), 
 ('V_0_2', [0, 1]), ('V_0_3', [0, 1]), 
 ('V_0_4', [(0, 0, 1, 1), (0, 1, 0, 1), (0, 1, 1, 0), (1, 0, 0, 1), (1, 0, 1, 0), (1, 1, 0, 0)]), 
 ('V_0_5', [0, 1]), 
 ('V_1_0', [(0, 1, 1), (1, 0, 1), (1, 1, 0)]), 
 ('V_1_1', [0, 1]), 
 ('V_1_2', [0, 1]), 
 ('V_1_3', [0, 1]), 
 ('V_1_4', [(0, 0, 0, 0, 1, 1), (0, 0, 0, 1, 0, 1), (0, 0, 0, 1, 1, 0), (0, 0, 1, 0, 0, 1), (0, 0, 1, 0, 1, 0), (0, 0, 1, 1, 0, 0), (0, 1, 0, 0, 0, 1), (0, 1, 0, 0, 1, 0), (0, 1, 0, 1, 0, 0), (0, 1, 1, 0, 0, 0), (1, 0, 0, 0, 0, 1), (1, 0, 0, 0, 1, 0), (1, 0, 0, 1, 0, 0), (1, 0, 1, 0, 0, 0), (1, 1, 0, 0, 0, 0)]), 
 ('V_1_5', [0, 1]), 
 ('V_2_0', [0, 1]), 
 ('V_2_1', [0, 1]), 
 ('V_2_2', [0, 1]), 
 ('V_2_3', [(0, 0, 0, 1, 1), (0, 0, 1, 0, 1), (0, 0, 1, 1, 0), (0, 1, 0, 0, 1), (0, 1, 0, 1, 0), (0, 1, 1, 0, 0), (1, 0, 0, 0, 1), (1, 0, 0, 1, 0), (1, 0, 1, 0, 0), (1, 1, 0, 0, 0)]), 
 ('V_2_4', [0, 1]), 
 ('V_2_5', [0, 1]), 
 ('V_3_0', [0, 1]), 
 ('V_3_1', [0, 1]), 
 ('V_3_2', [(0, 1, 1, 1, 1), (1, 0, 1, 1, 1), (1, 1, 0, 1, 1), (1, 1, 1, 0, 1), (1, 1, 1, 1, 0)]), 
 ('V_3_3', [0, 1]), 
 ('V_3_4', [(0, 0, 0, 0, 1, 1), (0, 0, 0, 1, 0, 1), (0, 0, 0, 1, 1, 0), (0, 0, 1, 0, 0, 1), (0, 0, 1, 0, 1, 0), (0, 0, 1, 1, 0, 0), (0, 1, 0, 0, 0, 1), (0, 1, 0, 0, 1, 0), (0, 1, 0, 1, 0, 0), (0, 1, 1, 0, 0, 0), (1, 0, 0, 0, 0, 1), (1, 0, 0, 0, 1, 0), (1, 0, 0, 1, 0, 0), (1, 0, 1, 0, 0, 0), (1, 1, 0, 0, 0, 0)]), 
 ('V_3_5', [0, 1]), 
 ('V_4_0', [0, 1]), 
 ('V_4_1', [(0, 0, 0, 0, 0, 1, 1), (0, 0, 0, 0, 1, 0, 1), (0, 0, 0, 0, 1, 1, 0), (0, 0, 0, 1, 0, 0, 1), (0, 0, 0, 1, 0, 1, 0), (0, 0, 0, 1, 1, 0, 0), (0, 0, 1, 0, 0, 0, 1), (0, 0, 1, 0, 0, 1, 0), (0, 0, 1, 0, 1, 0, 0), (0, 0, 1, 1, 0, 0, 0), (0, 1, 0, 0, 0, 0, 1), (0, 1, 0, 0, 0, 1, 0), (0, 1, 0, 0, 1, 0, 0), (0, 1, 0, 1, 0, 0, 0), (0, 1, 1, 0, 0, 0, 0), (1, 0, 0, 0, 0, 0, 1), (1, 0, 0, 0, 0, 1, 0), (1, 0, 0, 0, 1, 0, 0), (1, 0, 0, 1, 0, 0, 0), (1, 0, 1, 0, 0, 0, 0), (1, 1, 0, 0, 0, 0, 0)]), 
 ('V_4_2', [0, 1]), 
 ('V_4_3', [(0, 0, 0, 1, 1, 1), (0, 0, 1, 0, 1, 1), (0, 0, 1, 1, 0, 1), (0, 0, 1, 1, 1, 0), (0, 1, 0, 0, 1, 1), (0, 1, 0, 1, 0, 1), (0, 1, 0, 1, 1, 0), (0, 1, 1, 0, 0, 1), (0, 1, 1, 0, 1, 0), (0, 1, 1, 1, 0, 0), (1, 0, 0, 0, 1, 1), (1, 0, 0, 1, 0, 1), (1, 0, 0, 1, 1, 0), (1, 0, 1, 0, 0, 1), (1, 0, 1, 0, 1, 0), (1, 0, 1, 1, 0, 0), (1, 1, 0, 0, 0, 1), (1, 1, 0, 0, 1, 0), (1, 1, 0, 1, 0, 0), (1, 1, 1, 0, 0, 0)]), 
 ('V_4_4', [0, 1]), 
 ('V_4_5', [0, 1]), 
 ('V_5_0', [0, 1]), 
 ('V_5_1', [0, 1]), 
 ('V_5_2', [0, 1]), 
 ('V_5_3', [0, 1]), 
 ('V_5_4', [0, 1])]
```

O grafo terá de ser um dicionário em que as chaves são variáveis e os valores são listas de variáveis. Os nossos testes irão converter este dicionário para uma lista ordenada para ser possivel comparar automaticamente e a lista ordenada será a seguinte:
```
[('V_0_0', ['V_1_1']),
 ('V_0_1', ['V_0_2', 'V_1_1', 'V_1_2']),
 ('V_0_2', ['V_0_1']),
 ('V_0_3', ['V_0_4', 'V_1_4']),
 ('V_0_4', ['V_0_3', 'V_0_5', 'V_1_3', 'V_1_5']),
 ('V_0_5', ['V_0_4', 'V_1_4']),
 ('V_1_0', ['V_1_1', 'V_2_0', 'V_2_1']),
 ('V_1_1', ['V_0_0', 'V_0_1', 'V_1_0']),
 ('V_1_2', ['V_0_1', 'V_2_3']),
 ('V_1_3', ['V_0_4', 'V_1_4', 'V_2_3']),
 ('V_1_4', ['V_0_3', 'V_0_5', 'V_1_3', 'V_1_5', 'V_2_4', 'V_2_5']),
 ('V_1_5', ['V_0_4', 'V_1_4']),
 ('V_2_0', ['V_1_0']),
 ('V_2_1', ['V_1_0', 'V_3_2']),
 ('V_2_2', ['V_2_3', 'V_3_2']),
 ('V_2_3', ['V_1_2', 'V_1_3', 'V_2_2', 'V_2_4', 'V_3_3']),
 ('V_2_4', ['V_1_4', 'V_2_3', 'V_3_4']),
 ('V_2_5', ['V_1_4', 'V_3_4']),
 ('V_3_0', ['V_4_1']),
 ('V_3_1', ['V_3_2', 'V_4_1']),
 ('V_3_2', ['V_2_1', 'V_2_2', 'V_3_1', 'V_3_3', 'V_4_2']),
 ('V_3_3', ['V_2_3', 'V_3_2', 'V_3_4', 'V_4_3']),
 ('V_3_4', ['V_2_4', 'V_2_5', 'V_3_3', 'V_3_5', 'V_4_4', 'V_4_5']),
 ('V_3_5', ['V_3_4']),
 ('V_4_0', ['V_4_1']),
 ('V_4_1', ['V_3_0', 'V_3_1', 'V_4_0', 'V_4_2', 'V_5_0', 'V_5_1', 'V_5_2']),
 ('V_4_2', ['V_3_2', 'V_4_1', 'V_4_3']),
 ('V_4_3', ['V_3_3', 'V_4_2', 'V_4_4', 'V_5_2', 'V_5_3', 'V_5_4']),
 ('V_4_4', ['V_3_4', 'V_4_3']),
 ('V_4_5', ['V_3_4']),
 ('V_5_0', ['V_4_1']),
 ('V_5_1', ['V_4_1']),
 ('V_5_2', ['V_4_1', 'V_4_3']),
 ('V_5_3', ['V_4_3']),
 ('V_5_4', ['V_4_3'])]
```

A solução final deste puzzle pode ser mostrado com a função **fill_puzzle** apresentada anteriormente:
```
1 2 ~ ~ ~ ~ # 
1 @ @ ~ 2 ~ # 
~ ~ @ 4 @ @ # 
~ ~ 2 @ 3 ~ # 
2 2 ~ 2 ~ ~ # 
@ @ ~ @ ~ # # 
```

## Submissão

### Quizz

Cada grupo deve completar a implementação das funções pedidas e testá-las no link do quizz **Projecto 3: Minesweeper binário** que está na página da disciplina, introduzindo aí o vosso código.

Esse quizz é constituído por uma única pergunta. A implementação da função `minesweeper_CSP` é avaliada através de um conjunto de testes automáticos visíveis e mais alguns escondidos.

Este projecto é de correção automática e podem ir verificando o código fazendo check nos testes visíveis e invisíveis recebendo o feedback. Podem submeter as vezes que quiserem (independentemente do elemento do grupo), sendo a submissão com melhor nota a que será considerada.

### Ficheiro Python

Simultaneamente é necessário submeter na página da disciplina o ficheiro Python que contém todo o código utilizado para implementar a função `minsweeper_CSP`. **Os grupos que não tiverem submetido o ficheiro Python terão 0 valores de nota final** neste projecto, independentemente do resultado da avaliação automática. São livres de implementar outras funções dentro ou fora da função `minesweeper_CSP` e poderão importar outros módulos desde que seja reconhecido pelo Moodle.

O ficheiro Python a submeter deve chamar-se **SI-proj3-XX.py** que que substituem XX pelo número do grupo. Basta um dos elementos do grupo submeter o ficheiro.

### Dúvidas

Todas as dúvidas que achem que são do interesse geral devem ser submetidas para o fórum da disciplina no Moodle. Dúvidas mais privadas, que envolvam resultados de testes escondidos e código, devem ser dirigidas apenas para haidos@ciencias.ulisboa.pt.

### Prazo de entrega

O prazo de entrega é 28 de Abril às 23:59. **Não se aceitam entregas de projetos por email.**